In [ ]:
import sys
import os
import json
import sklearn
import pandas as pd
import numpy as np
from openai import OpenAI
from tqdm.auto import tqdm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Complaints.xlsx to Complaints.xlsx
Saving CustomerData.xlsx to CustomerData.xlsx


#RAG on Complaint

We have utilized RAG to analyse the complaints dataset and better understand the customers' main concerns. To do so, we have utilised the openai api and had gpt access the file and analyse it throguh a prompt

In [ ]:
### ======== CONFIGURATION ============
client = OpenAI(api_key="")

In [ ]:
complaints_df = pd.read_excel("Complaints.xlsx")
complaints_df.rename(columns=lambda x: x.strip().lower(), inplace=True)
assert "complaint" in complaints_df.columns, "Missing 'complaint' column"
assert "customerid" in complaints_df.columns, "Missing 'customerid' column"

In [ ]:
#========EXTRACT MAIN THEMES VIA GPT ============

all_complaints_text = "\n".join(complaints_df['complaint'].dropna().astype(str).tolist())

def split_into_chunks(text, max_tokens=2000):
    """Split a long text into smaller chunks that fit within the model's token limit."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_chunk_len = 0
    for word in words:
        current_chunk_len += len(word) + 1  # +1 for space
        if current_chunk_len <= max_tokens:
            current_chunk.append(word)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_chunk_len = len(word) + 1
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

complaints_chunks = split_into_chunks(all_complaints_text, max_tokens=2000)

def extract_themes_from_chunk(chunk):
    prompt = f"""
You are a customer service representative at ABC TelCo, analyzing verbatim customer complaints to uncover the company’s most pressing service issues.

1. **Input Format**
   You will receive a numbered list of individual complaints, each on its own line. A single line may describe **multiple distinct issues** (e.g., “The internet drops and the bill was wrong this month” contains two separate complaints). For example:
   1. “The internet is constantly dropping during peak hours, making streaming impossible.”
   2. “My monthly bill increased by \$20 without any notice or explanation.”
   3. “I was on hold for nearly an hour and the agent couldn’t resolve my issue.”
   …and so on.

2. **Your Task**
   - Read *all* complaints.
   - Identify the **distinct, high-level themes** that capture why customers are unhappy.
   - One complaint line may map to **multiple themes**.
   - Merge similar ideas (e.g. “slow internet” + “frequent outages” → “poor internet reliability”).

3. **Output Format**
   - Return a **single bullet list** of theme labels, **sorted by importance** (most frequent/critical first).
   - Use concise, business-friendly wording (e.g. `• Billing transparency issues`, `• Poor internet reliability`).
   - Do **not** include complaint text, counts, or commentary—just the theme names.

---

Here are the complaints to analyze:
{chunk}
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant trained to identify complaint themes."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=300
    )
    # Extract the content from the response
    themes_text = response.choices[0].message.content.strip()
    return themes_text

all_extracted_themes = []
for chunk in complaints_chunks:
    themes_text = extract_themes_from_chunk(chunk)

    # Process the themes output: remove any bullet symbols and whitespace
    extracted_themes = []
    for line in themes_text.splitlines():
        clean_line = re.sub(r'^[-*•]\s*', '', line).strip()
        if clean_line:
            extracted_themes.append(clean_line)
    all_extracted_themes.extend(extracted_themes)

# Remove duplicates to get the distinct set of themes
unique_themes = list(set(all_extracted_themes))
print("Extracted Themes (all complaints):")
print(unique_themes)

Extracted Themes (all complaints):
['Lack of bundled options or discounts', 'Billing process discrepancies and inconvenience', 'Poor customer support', 'Unhelpful customer service', 'Slow customer service response times', 'Unresponsive customer service', 'Unreliable phone service', 'Lack of loyalty discounts', 'Ineffective tech support for streaming problems', 'Billing transparency issues', 'Lack of online security options', 'High monthly charges compared to service quality', 'Outdated payment process', 'Ineffective security services', 'High monthly charges without value', 'Difficulty managing services and billing', 'Difficulty in canceling service', 'Streaming service buffering', 'Subpar phone call quality', 'Confusing billing system and payment delays', 'Lack of online security features', 'High charges for perceived low value', 'Inadequate customer service and tech support', 'Subpar streaming service performance', 'Limited internet service options', 'Poor streaming quality', 'Unsatis

*After the themes have been extracted from all the complaints we used an LLM to find the common and unique themes accross the various rows of the data*

In [ ]:
theme_defs = {
    "Poor Internet Speed": (
        "slow, inconsistent connectivity with frequent disconnections and long downtimes "
        "affecting browsing, gaming, and other online activities"
    ),
    "Poor Streaming Service Quality": (
        "persistent buffering, lag, freezing, and interruptions during streaming of "
        "TV shows, movies, or video content"
    ),
    "Billing Issues": (
        "discrepancies and lack of transparency in billing statements including unexpected "
        "high fees, incorrect charges, or unclear breakdowns"
    ),
    "Payment Issues": (
        "problems with processing payments, failed transactions, difficulties with payment "
        "methods, or issues with automatic payments"
    ),
    "Poor Customer Support": (
        "long wait times, difficulty reaching a support representative, unhelpful or inaccurate "
        "responses, and unresolved issues"
    ),
    "Poor Technical Support": (
        "delays in reaching technical support, prolonged wait times for assistance, and "
        "insufficient or ineffective technical guidance"
    ),
    "Poor Phone Service Quality": (
        "dropped calls, poor voice clarity, static interference, and inconsistent phone line "
        "performance"
    ),
    "Limited Device Protection": (
        "inadequate protection plans for devices, missing backup services for data, or "
        "insufficient device security features offered"
    ),
    "Online Security Issues": (
        "lack of adequate security features or vulnerabilities in online protection against "
        "malware, phishing, or other cyber threats"
    ),
    "Contract Issues": (
        "inflexible contract terms with limited options in service agreements, early "
        "termination fees, or lack of clarity in contract conditions"
    ),
    "Cancellation Issues": (
        "complications in canceling or terminating service, lengthy processes, or disputes "
        "over cancellation fees"
    ),
    "Limited Service Options": (
        "few choices in bundled internet and streaming services, lack of customization options, "
        "or insufficient loyalty rewards for long-term subscribers"
    ),
    "Customer Loyalty": (
        "lack of recognition, rewards, or incentives for long-term customers, feeling undervalued "
        "compared to new subscribers"
    ),
    "High Monthly Charge": (
        "excessive fees or costs (explicit phrases like 'high monthly charges', 'expensive', "
        "'overcharged') that do not align with the perceived value of services received"
    ),
}

# flatten names for ordering
short_theme_names = list(theme_defs.keys())

def classify_with_openai_rich(text: str) -> dict:
    system = {
        "role": "system",
        "content": (
            "You are a customer-service analytics assistant. For each complaint text, "
            "you will decide **for each** of the 14 provided themes whether the customer "
            "is complaining about it. Multiple themes may apply. "
            "Return **only** a JSON object mapping each theme name (exactly) to 1 or 0."
        )
    }
    # build the user prompt with definitions
    user_lines = ["Here are the themes and what each means:"]
    for name, desc in theme_defs.items():
        user_lines.append(f"- **{name}**: {desc}")
    user_lines.append("\nComplaint:")
    user_lines.append(text)
    user_lines.append("\nNow return a JSON only, e.g.:")
    user_lines.append('{ "Poor Internet Speed": 1, "Billing Issues": 0, … }')

    user = {"role": "user", "content": "\n".join(user_lines)}

    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[system, user],
        temperature=0.0,
        max_tokens=200
    )
    return json.loads(resp.choices[0].message.content.strip())

# then loop exactly as before:
results = []
for txt in tqdm(df["complaint"], desc="Classifying complaints"):
    try:
        results.append(classify_with_openai_rich(txt))
    except Exception:
        # fallback to all zeros
        results.append({t: 0 for t in short_theme_names})

labels_df = pd.DataFrame(results)
df = pd.concat([df, labels_df], axis=1)
df.to_csv("Complaints_with_OpenAI_Themes.csv", index=False)
print("✅ Done.")



Classifying complaints:   0%|          | 0/1605 [00:00<?, ?it/s]

✅ Done.
